# 장르 속성을 이용한 영화 콘텐츠 기반 필터링
장르 칼럼 값의 유사도를 비교한 뒤 높은 평점을 가지는 영화를 추천

# 데이터 로딩 및 가공 

In [2]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

movies = pd.read_csv('./tmdb-movie-metadata/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [3]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords',]]

In [4]:
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [5]:
print(movies_df['genres'])

0       [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {...
1            [{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]
2              [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]
3       [{"id": 28, "name": "Action"}, {"id": 80, "name": "Crime"}, {"id": 18, "name": "Drama"}, {"id": ...
4       [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 878, "name": "Science Fic...
                                                       ...                                                 
4798            [{"id": 28, "name": "Action"}, {"id": 80, "name": "Crime"}, {"id": 53, "name": "Thriller"}]
4799                                       [{"id": 35, "name": "Comedy"}, {"id": 10749, "name": "Romance"}]
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"...
4801                        

In [6]:
'''
import ast



str_dict = "{'a': 3, 'b': 5}"

print type(str_dict)           # <type 'str'>



convert_dict = ast.literal_eval(str_dict)



print type(convert_dict)   # <type 'dict'>

print convert_dict['a']      #  3

print convert_dict['b']      #  5

'''
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [7]:
print(movies_df['genres'])

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
1            [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]
2              [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name': 'Drama'}, {'id': ...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, 'name': 'Science Fic...
                                                       ...                                                 
4798            [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name': 'Thriller'}]
4799                                       [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'...
4801                        

In [8]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x]) #리스트 내 여러개의 딕셔너리의 'name' key에 해당하는 값을 찾아 이를 리스트 객체로 변환한다.
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


# 장르별 유사도 측정
- 문자열로 변환된 genres 칼럼을 Count 기반으로 피처벡터화 변환한다.
- genres 문자열을 피처벡터 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교한다.  

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x)) #리스트에서 문자열로
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [10]:
movies_df['genres_literal'][:1]

0    Action Adventure Fantasy Science Fiction
Name: genres_literal, dtype: object

일반적으로 코사인 유사도는 벡터의 크기가 중요하지 않을때 거리를 측정하기 위한 메트릭으로 사용된다.  
예를 들어 단어의 포함 여부로 문서의 유사 여부를 판단한다고 할때 ‘science’라는 단어가 2번 보다 1번 문서에 더 많이 포함되어 있다면  
1번 문서가 과학 문서라고 추측할 수 있을 것이다. 그러나, 만약 1번 문서가 2번 문서 보다 훨씬 더 길다면 공정하지 않은 비교가 된다.  
이때 코사인 유사도는 이 문제를 바로 잡아줄 수 있다.

즉, 길이를 정규화해 비교하는 것과 유사하다고 할 수 있으며 이 때문에 텍스트 데이터를 처리하는 메트릭으로 주로 사용된다.  
주로 데이터 마이닝이나 정보 검색information retrieval에서 즐겨 사용된다.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
'''
Compute cosine similarity between samples in X and Y.

Cosine similarity, or the cosine kernel, computes similarity as the normalized dot product of X and Y:

K(X, Y) = <X, Y> / (||X||*||Y||)

http://docs.likejazz.com/cosine-sim/
'''
genre_sim = cosine_similarity(genre_mat)

In [13]:
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [17]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]#인덱스임을 주의
print(genre_sim_sorted_ind[0])# 0번 레코드가 가장 유사도가 높은 것은 3494

[   0 3494  813 ... 3038 3037 2401]


# 장르 콘텐츠 필터링 이용한 영화 추천


In [15]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title']== title_name]
    title_index = title_movie.index.values

    similar_indexes = sorted_ind[title_index,:(top_n)]

    #print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [16]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


단순히 장르 유사도로 추출했을 때, 평점이 나쁜 영화도 끼어있음.

좀 더 많은 후보군을 선정한 뒤, 영화의 평점에 따라 필터링해서 최종 추천해보자

In [26]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending = False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


소수의 관객이 특정 영화에 높은 점수 부여 (왜곡된 데이터)
    -> 새로운 평가 방식 weighted rating 으로 평가
    
- (v/(v+m)) * R + (m/(v+m)) * c  
v: 개별 영화에 평점을 투표한 횟수(vote_count)  
m: 평점을 부여하기 위한 최소 투표 횟수  
R: 개별 영화에 대한 평균 평점(vote_average)    
c: 전체 영화에 대한 평균 평점(movies_df['vote_average'].mean)    

m값을 높이면 평점 투표 횟수가 적은 영화에 더 적은 가중 평점을 부과함.

In [31]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로
print('C:',round(C,3),'m:', round(m,3))

C: 6.092 m: 370.2


In [39]:
percentile = 0.6
m = movies['vote_count'].quantile(percentile)
C = movies['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m)) * R + (m/(v+m)) * C)

movies_df['weighted_vote'] = movies.apply(weighted_vote_average, axis = 1)

In [40]:
movies_df[['title','vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending = False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [47]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_indexes = sorted_ind[title_index, :(top_n*2)]#장르 유사 후보군을 두배로 늘림
    similar_indexes = similar_indexes.reshape(-1)
    
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending = False)[:top_n] #가중 평점을 적용하고, 10개 추출

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]
    

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
